<a href="https://colab.research.google.com/github/Prabhsimran1313/CVD_Disease_Prediction_using_ML/blob/main/CVD__Disease_Prediction_ML_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries

In [ ]:
!pip install kmodes!pip install tensorflow

In [ ]:
import numpy as npimport pandas as pdfrom sklearn import metrics, preprocessingfrom sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_scorefrom sklearn.model_selection import train_test_split, cross_val_score, GridSearchCVfrom sklearn.linear_model import LogisticRegressionfrom sklearn.ensemble import RandomForestClassifierfrom sklearn.neural_network import MLPClassifierfrom sklearn.svm import SVCfrom kmodes.kmodes import KModesfrom sklearn.metrics import classification_reportfrom sklearn.preprocessing import LabelEncoderimport tensorflow as tffrom tensorflow.keras.models import Sequentialfrom tensorflow.keras.layers import Dense, Dropoutfrom tensorflow.keras.callbacks import EarlyStoppingfrom sklearn.metrics import classification_report, accuracy_score, confusion_matriximport matplotlib.pyplot as pltimport seaborn as sns

In [ ]:
from google.colab import drivedrive.mount('/content/drive')

### Reading the data

In [ ]:
# read the data and show first 5 rowsdf = pd.read_csv("/content/drive/MyDrive/Datasets ML/cardio_train.csv",sep=";")df.head(5)

In [ ]:
df.info()

### Initial Inspection

In [ ]:
print(df.isnull().sum())

In [ ]:
#drop iddf = df.drop('id', axis=1)

### Exploratory Data Analysis (EDA)

In [ ]:
# Count the number of samples per classcardio_counts = df['cardio'].value_counts()print("Cardio Class Counts:\n", cardio_counts)# Plot the class distributionsns.countplot(x='cardio', data=df)plt.title('Distribution of Target Variable (Cardiovascular Disease)')plt.xlabel('Cardio (0 = No Disease, 1 = Disease)')plt.ylabel('Count')plt.show()

In [ ]:
# Pairplot of selected features against targetsns.pairplot(df[['age', 'weight', 'ap_hi', 'ap_lo', 'cardio']], hue='cardio')plt.suptitle("Pairplot of Features by Cardio", y=1.02)plt.show()

In [ ]:
# Select relevant columnsoutlier_cols = ['height', 'weight', 'ap_hi', 'ap_lo']# Melt the DataFrame to long formatdf_melted = df[outlier_cols].melt(var_name='Feature', value_name='Value')# Create a single boxplotplt.figure(figsize=(10, 6))sns.boxplot(x='Feature', y='Value', data=df_melted)plt.title('Combined Boxplot for Outlier Detection')plt.grid(True)plt.show()

### Removing Outliers

In [ ]:
df.drop(df[(df['height'] > df['height'].quantile(0.975)) | (df['height'] < df['height'].quantile(0.025))].index,inplace=True)df.drop(df[(df['weight'] > df['weight'].quantile(0.975)) | (df['weight'] < df['weight'].quantile(0.025))].index,inplace=True)df.drop(df[(df['ap_hi'] > df['ap_hi'].quantile(0.975)) | (df['ap_hi'] < df['ap_hi'].quantile(0.025))].index,inplace=True)df.drop(df[(df['ap_lo'] > df['ap_lo'].quantile(0.975)) | (df['ap_lo'] < df['ap_lo'].quantile(0.025))].index,inplace=True)len(df)

In [ ]:
df[df['ap_lo']> df['ap_hi']].shape[0] # To check number of cases where diastolic pressure is higher than systolic?

In [ ]:
# After removing outliersdf.describe()

### Feature Engineering

In [ ]:
df['age'] = (df['age'] / 365).round().astype('int')  # Transformation: Converting age from days to yearsprint(df.head())

In [ ]:
# Categorizing features# # Define the bin edges and labelsage_edges = [30, 35, 40, 45, 50, 55, 60, 65]age_labels = [0, 1, 2, 3, 4, 5, 6]#  bin in  5 years spandf['age_group'] = pd.cut(df['age'], bins=7, labels=range(7), include_lowest=True,right=True)df.head()

In [ ]:
df['bmi'] = df['weight']/((df['height']/100)**2)df.head()bmiMin = int(df['bmi'].min())bmiMax = int(df['bmi'].max())print(bmiMin, bmiMax)df['bmi'] = pd.cut(df['bmi'], bins=6, labels=range(6), right=True, include_lowest=True)df.head()df["bmi"].value_counts(normalize=True)

In [ ]:
df['map'] = ((2* df['ap_lo']) + df['ap_hi']) / 3mapMin = int(df['map'].min())mapMax = int(df['map'].max())print(mapMin, mapMax)df['map'] = pd.cut(df['map'], bins=6, labels=range(6), right=True, include_lowest=True)df.head()

In [ ]:
null_rows = df[df.isnull().any(axis=1)]print("Rows with null values:")print(null_rows)

In [ ]:
# Drop features We only need categorical datadf_og=dfdf=df.drop(['height','weight','ap_hi','ap_lo','age'],axis=1)df.head()

### Encoding

In [ ]:
le = preprocessing.LabelEncoder()df = df.apply(le.fit_transform)df.describe()

### Clustering (KModes)

In [ ]:
cost = []num_clusters = range(1,6) # 1 to 5for i in list(num_clusters):    kmode = KModes(n_clusters=i, init = "Huang", n_init = 5, verbose=0,random_state=1)    kmode.fit_predict(df)    cost.append(kmode.cost_)

In [ ]:
# Clusters graphplt.plot(num_clusters, cost, 'bo-')plt.xlabel('num_clusters')plt.ylabel('Cost')plt.title('Elbow Method For Optimal Number of Clusters')plt.show()

In [ ]:
# Building KModes modelkm = KModes(n_clusters=2, init = "Huang", n_init = 5,random_state=1)clusters = km.fit_predict(df)clusters

In [ ]:
# Adding clusters column in DFdf.insert(0,"clusters",clusters,True)df.head()

### Cluster-Based Visualizations

In [ ]:
# Correlation Matrixplt.figure(figsize=(10, 8))# Draw correlation matrixsns.heatmap(df.corr(), annot=True, cmap='vlag', fmt=".2f", linewidths=.5)# Show the figureplt.title('Correlation Matrix')plt.show()

In [ ]:
# Distribution of cardio in clusterssns.countplot(x='clusters', hue='cardio', data=df)plt.title('Distribution of Cardiovascular Disease within Clusters')plt.show()

In [ ]:
x = df.drop(['cardio','gender','alco'], axis=1)y = df['cardio']x.head()

### Train-Test Split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=1)

In [ ]:
x_train.info()

### Model training

### Random Forest

In [ ]:
from sklearn.metrics import accuracy_scorerfModel = RandomForestClassifier(    n_estimators=200,    max_depth=20,    min_samples_split=5,    min_samples_leaf=2,    max_features='sqrt',    random_state=1)# Train the modelrfModel.fit(x_train, y_train)# Predict on test datarf_pred = rfModel.predict(x_test)# Accuracyrf_accuracy = accuracy_score(y_test, rf_pred) * 100print(f"Random Forest Accuracy: {rf_accuracy:.2f}%")

### Performance Evaluation – Random Forest

In [ ]:
classification_report_str = classification_report(y_test, rf_pred)print("Classification Report for RF:\n", classification_report_str)

In [ ]:
# Confusion Matrixcm = confusion_matrix(y_test, rf_pred)sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')plt.title("Confusion matrix: RF")plt.xlabel("Predicted")plt.ylabel("Actual")plt.show()

In [ ]:
# Get predicted probabilities for positive class (1)rf_probs = rfModel.predict_proba(x_test)[:, 1]# Compute ROC curve and AUC scorefpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs)auc_rf = roc_auc_score(y_test, rf_probs)# Plot ROC curveplt.figure(figsize=(8, 6))plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {auc_rf:.2f})", color='blue')plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')plt.xlabel("False Positive Rate")plt.ylabel("True Positive Rate")plt.title("ROC Curve - Random Forest")plt.legend(loc="lower right")plt.grid(True)plt.show()

### ANN Model

In [ ]:
# Define ANN architecturemodel = Sequential()model.add(Dense(128, input_dim=x_train.shape[1], activation='relu'))model.add(Dropout(0.3))  # prevent overfittingmodel.add(Dense(64, activation='relu'))model.add(Dropout(0.2))model.add(Dense(32, activation='relu'))model.add(Dense(1, activation='sigmoid'))  # Binary classification# Compile modelmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Use early stopping to avoid overfittingearly_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)# Trainhistory = model.fit(    x_train, y_train,    validation_split=0.2,    epochs=100,    batch_size=32,    callbacks=[early_stop],    verbose=1)

In [ ]:
# Predict on test sety_pred_prob = model.predict(x_test)y_pred = (y_pred_prob > 0.5).astype(int)# Accuracyann_accuracy = accuracy_score(y_test, y_pred) * 100print(f"ANN Accuracy: {ann_accuracy:.2f}%")

### Performance Evaluation – ANN

In [ ]:
# Reportprint("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Confusion Matrixcm = confusion_matrix(y_test, y_pred)sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')plt.title("Confusion Matrix - ANN")plt.xlabel("Predicted")plt.ylabel("Actual")plt.show()

In [ ]:
fpr_ann, tpr_ann, _ = roc_curve(y_test, y_pred_prob)auc_ann = roc_auc_score(y_test, y_pred_prob)plt.figure(figsize=(8, 6))plt.plot(fpr_ann, tpr_ann, label=f"ANN (AUC = {auc_ann:.2f})", color='green')plt.plot([0, 1], [0, 1], 'k--', label='ANN')plt.xlabel("False Positive Rate")plt.ylabel("True Positive Rate")plt.title("ROC Curve - ANN")plt.legend(loc="lower right")plt.grid(True)plt.show()

### Performance Comparison

In [ ]:
print(f"Random Forest Accuracy: {rf_accuracy:.2f}% | AUC: {auc_rf:.2f}")print(f"ANN Accuracy: {ann_accuracy:.2f}%       | AUC: {auc_ann:.2f}")

In [ ]:
plt.figure(figsize=(8, 6))plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {auc_rf:.2f})", color='blue')plt.plot(fpr_ann, tpr_ann, label=f"ANN (AUC = {auc_ann:.2f})", color='green')plt.plot([0, 1], [0, 1], 'k--')plt.xlabel("False Positive Rate")plt.ylabel("True Positive Rate")plt.title("ROC Curve Comparison")plt.legend(loc="lower right")plt.grid(True)plt.show()